# pset 09: the return of the ten arcs

# 1. write a simulator as a positive control

In [1]:
# import necessary packages
import numpy as np
import string
from scipy.special import logsumexp

# create a function to simulate read counts given tau and length values
def simulate(T_i, L_i, N):
    counts = 10*[0]
    
    # create a list of the 10 transcript indices
    index = [i for i in range(10)]
    
    # convert transcript to nucleotide abundances
    nu_i = [T_i[i]*L_i[i] for i in range(10)]
    nu_i = [i / sum(nu_i) for i in nu_i]
    
    # loop for desired number of read counts
    for j in range(N):
        
        # choose a transcript with taus as probability distribution
        g = np.random.choice(index, p=nu_i)
        
        # randomly choose a segment present in that transcript
        l = np.random.randint(0, L_i[g])
        s = index[(g+l) % 10]
        
        # increment that read count's total
        counts[s] += 1
        
    return counts

In [11]:
# initialize transcript lengths, random taus, number of read counts
L_i = [4, 2, 3, 4, 4, 3, 2, 2, 3, 3]
T_i = [np.random.randint(100) for i in range(10)]
T_i = [i / sum(T_i) for i in T_i]
N = 1000000

# generate test dataset
sim_counts = simulate(T_i, L_i, N)
sim_counts

[128605, 162065, 111215, 76593, 51008, 49216, 90492, 135499, 101172, 94135]

# 2. calculate the log likelihood

In [3]:
# create function to compute negative log likelihood
def nll(counts, T_i, L_i):
    
    # create list of transcript/segment indicies
    index = [i for i in range(10)]
    
    # calculate nu values from tau values
    nu_i = [T_i[i]*L_i[i] for i in range(10)]
    nu_i = [i / sum(nu_i) for i in nu_i]
    
    # create list of lists for the segments present in each transcript
    arcs = []
    for i in range(10):
        
        # if transcript is 9 or 10, wrap around the circular arc locus
        if i == 8 or i == 9:
            s_list = index[i:] + index[0:((i+L_i[i]) % 10)]
        else:
            s_list = index[i:i+L_i[i]]
        arcs.append(s_list)
    
    # iterate through reads, calculate likelihood
    ll_tot = 0
    for read_index in range(10):
        ll = 0
        for gene_index in range(10):
            
            # if segment present in transcript, compute probability
            if read_index in arcs[gene_index]:
                ll  += nu_i[gene_index] / L_i[gene_index]
        
        # multiply by number of counts (adding up log probabilities)
        ll_tot += np.log(ll) * counts[read_index]
    
    return -ll_tot

In [4]:
# lestrade's approach (code taken from pset)

def lestrade(counts, T, L):
    r = counts
    T = 10
    S = T    # S = R = T : there are T transcripts (Arc1..Arc10), S segments (A..J), R reads (a..j)
    R = T
    Slabel   = list(string.ascii_uppercase)[:S]               # ['A'..'J']        : the upper case labels for Arc locus segments 
    Tlabel   = [ "Arc{}".format(d) for d in range(1,T+1) ]    # ['Arc1'..'Arc10'] : the labels for Arc transcript isoforms
    Rlabel   = list(string.ascii_lowercase)[:T]               # ['a'..'j']        : lower case labels for reads


    # Count how often each segment A..J is used in the isoforms i
    # We'll use that to split observed read counts across the isoforms
    # that they might have come from.
    #
    segusage = np.zeros(S).astype(int)
    for i in range(T):
        for j in range(i,i+L[i]): 
            segusage[j%S] += 1


    # Naive analysis:
    #
    c  = np.zeros(T)
    for i in range(T):
        for k in range(i,i+L[i]):
            c[i] += (1.0 / float(segusage[k%S])) * float(r[k%S])  # For each read k, assume read k-> segment j,
                                                                  # and assign 1/usage count to each transcript
                                                                  # that contains segment j.
    Z       = np.sum(c)
    est_nu  = np.divide(c, Z)       # nucleotide abundance

    est_tau = np.divide(est_nu, L)  # convert to TPM, transcript abundance
    est_tau = np.divide(est_tau, np.sum(est_tau))
        
    return est_tau

In [12]:
# run lestrade's appraoch to find his estimated taus
est_tau = lestrade(sim_counts, 10, L_i)
Tlabel   = [ "Arc{}".format(d) for d in range(1,11) ]

# print both sets of tau values for comparison
print('Lestrade tau estimates:')
for i in range(10):
    print ("{0:10s}  {1:5.3f}".format(Tlabel[i], est_tau[i]))

print('\ntrue taus:')    
for i in range(10):
    print ("{0:10s}  {1:5.3f}".format(Tlabel[i], T_i[i]))

Lestrade tau estimates:
Arc1        0.116
Arc2        0.132
Arc3        0.077
Arc4        0.059
Arc5        0.065
Arc6        0.071
Arc7        0.082
Arc8        0.123
Arc9        0.136
Arc10       0.139

true taus:
Arc1        0.093
Arc2        0.169
Arc3        0.040
Arc4        0.074
Arc5        0.024
Arc6        0.034
Arc7        0.113
Arc8        0.197
Arc9        0.076
Arc10       0.179


In [6]:
# compare negative log likelihoods
ll_me = nll(sim_counts, T_i, L_i)
ll_lestrade = nll(sim_counts, est_tau, L_i)

print('my negative log likelihood:', ll_me)
print("Lestrade's negative log likelihood:", ll_lestrade)
print('difference in log likelihoods:', ll_lestrade-ll_me)

my negative log likelihood: 2090723.9074411509
Lestrade's negative log likelihood: 2096553.2595191002
difference in log likelihoods: 5829.3520779493265


We can see that Lestrade's tau value estimates are pretty bad in comparison to the true values. For example, Lestrade predicts Arcs 3 and 9 to have almost twice the probability than they actually do. To further show this, we compute the negative log likelihoods for each set of values. Comparing, we can see that the true taus produce a likelihood almost 6000 smaller than Lestrade's taus, meaning that his parameters are not a great fit for the data. 

# 3. estimate isoform abundances by EM

In [7]:
# expectation step
def expectation(counts, nu_i, L_i):
    
    # create list of transcript/segment indicies
    index = [i for i in range(10)]
    
    # create list of lists for the segments present in each transcript
    arcs = []
    for i in range(10):
        if i == 8 or i == 9:
            s_list = index[i:] + index[0:((i+L_i[i]) % 10)]
        else:
            s_list = index[i:i+L_i[i]]
        arcs.append(s_list)
    
    # create an i by j array that determines if segment j is in transcript i
    # 1 if present, 0 otherwise
    in_array = []
    for i in range(10):
        in_gene = []
        for j in range(10):
            if j in arcs[i]:
                in_gene.append(1)
            else:
                in_gene.append(0)
        in_array.append(in_gene)
    
    # multiply the element in i, j by the ith nu and 1/length values
    # (compute probability of each segment given each transcript)
    prob_seq = np.array([[num * nu_i[i] / L_i[i] for num in in_array[i]] for i in range(10)])
    
    # normalize over all transcripts (sum over columns)
    prob_norm = [[prob / sum(prob_seq[:,i]) for prob in prob_seq[:,i]] for i in range(10)]
    
    # multiply by number of counts and sum for each transcript
    exp_counts = [sum([prob_norm[j][i] * counts[j] for j in range (10)]) for i in range (10)]
        
    return exp_counts


# maximization step
def maximization(exp_counts):
    # convert expected counts to probabilities by dividing by sum
    new_nu_i = [count/sum(exp_counts) for count in exp_counts]
    return new_nu_i


# run EM
def EM(counts, L_i):
    # initialize random taus, normalize
    T_i = [np.random.randint(1, 100) for i in range(10)]
    T_i = [i / sum(T_i) for i in T_i]
    
    # convert taus to nus, normalize
    nu_i = [T_i[i]*L_i[i] for i in range(10)]
    nu_i = [i / sum(nu_i) for i in nu_i]
    
    # initialize log likelihood values
    old_ll = 0
    new_ll = 1
    new_nu_i = nu_i
    
    # run until log likelihood changes by 0.00001 or less
    while abs(new_ll-old_ll) > 0.00001:
        
        # run expectation, then maximization to update nus
        exp_counts = expectation(counts,new_nu_i,L_i)
        new_nu_i = maximization(exp_counts)
        
        # compute taus from nus to calculate log likelihood
        tau_i = [new_nu_i[i] / L_i[i] for i in range(10)]
        tau_i = [tau / sum(tau_i) for tau in tau_i]
        old_ll = new_ll
        new_ll = nll(counts,tau_i,L_i)  
    
    return new_ll, tau_i

In [8]:
# run EM on counts data given in pset, print tau estimates
L_i = [4, 2, 3, 4, 4, 3, 2, 2, 3, 3]
data_counts = [89357,69196,87852,99854,57369,85715,197730,213016,61271,38640]

best_ll, best_tau = EM(data_counts, L_i)

print('my tau estimates:')
for i in range(10):
    print("{0:10s}".format(Tlabel[i]), best_tau[i])

my tau estimates:
Arc1       0.1414689644906891
Arc2       0.019259730963126224
Arc3       0.08436879588987034
Arc4       0.05277139327728457
Arc5       0.022906837918719985
Arc6       0.1634626848002875
Arc7       0.31249812260394344
Arc8       0.09545045743603983
Arc9       0.07548779053274306
Arc10      0.03232522208729613


In [9]:
# run lestrade's method on same data, print tau estimates
les_tau = lestrade(data_counts, 10, L_i)
les_ll = nll(data_counts, les_tau, L_i)

print('Lestrade tau estimates:')
for i in range(10):
    print ("{0:10s}  {1:5.3f}".format(Tlabel[i], est_tau[i]))

Lestrade tau estimates:
Arc1        0.062
Arc2        0.051
Arc3        0.098
Arc4        0.143
Arc5        0.153
Arc6        0.157
Arc7        0.148
Arc8        0.097
Arc9        0.048
Arc10       0.044


In [10]:
# compare negative log likelihoods
print('my negative log likelihood:', best_ll)
print("Lestrade's negative log likelihood:", les_ll)
print('difference in log likelihoods:', les_ll-best_ll)

my negative log likelihood: 2165610.3540291633
Lestrade's negative log likelihood: 2190089.552613845
difference in log likelihoods: 24479.198584681842


Lestrade's conclusion that all 10 mRNA transcripts are expressed at the same level seems incorrect. Using EM, we have calculated transcript abundance values that produce a smaller log likelihood value (by roughly 25000), meaning they are much closer to the true values. The two most abundant transcripts are Arcs 6 and 7, which account for almost half of the population. The two least abundant transcripts are Arcs 2 and 5, which account for less than 5% of the population.

Intuitively, Lestrade's method doesn't seem to be as accurate, since it assumes each read maps to the transcripts in which it could be found with equal probability. However, this doesn't take into account that the transcript abundance values should influence the probability a certain read belongs to a certain transcript. Thus, it makes sense that EM produces better estimates. 